### Generate dataset

### Analysis

In [ ]:
from importlib import reload
import embeding_visualization
reload(embeding_visualization)
from embeding_visualization import get_embeddings

from sklearn.decomposition import PCA
import numpy as np

import pandas as pd
import ast

# Load the CSV file
df = pd.read_csv('game_results.csv')

# Assuming the columns are named 'Past words player 1' and 'Past words player 2' but select only the first row
words_player1 = ast.literal_eval(df['Past words player 1'].iloc[3])
words_player2 = ast.literal_eval(df['Past words player 2'].iloc[3])

embeddings_1 = get_embeddings(words_player1)
embeddings_2 = get_embeddings(words_player2)

In [ ]:
print(words_player1)
print(words_player2)
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Calculate Euclidean distance between corresponding points
distances = [euclidean(embeddings_1[i], embeddings_2[i]) for i in range(len(embeddings_1))]

# Plot the evolution of distances
plt.figure(figsize=(10, 6))
plt.plot(distances, label='Distance between embeddings', color='blue', marker='o')
plt.xlabel('Index')
plt.ylabel('Euclidean Distance')
plt.title('Evolution of Distance between Embeddings')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# merge the embeddings
embeddings = embeddings_1 + embeddings_2

# Use PCA to reduce to 3 dimensions
pca = PCA(n_components=3)
pca_result = pca.fit(np.array(embeddings))
reduced_embeddings_1 = pca.transform(embeddings_1)
reduced_embeddings_2 = pca.transform(embeddings_2)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

# Function to get color from a Plotly colormap
def get_color_from_px_cmap(cmap_name, value):
    """Get the color from a Plotly Express colormap given a value between 0 and 1."""
    cmap = px.colors.get_colorscale(cmap_name)
    scaled_value = value * (len(cmap) - 1)  # Scale value to the colormap range
    index = int(np.floor(scaled_value))
    return cmap[index][1]  # Extract the color

# Function to create gradient lines using Plotly Express colormaps
def create_px_colormap_gradient_lines(embeddings, num_points, cmap_name):
    lines = []
    for i in range(num_points - 1):
        factor = i / (num_points - 1)  # Normalized position for gradient
        color = get_color_from_px_cmap(cmap_name, factor)
        line = go.Scatter3d(
            x=[embeddings[i, 0], embeddings[i + 1, 0]],
            y=[embeddings[i, 1], embeddings[i + 1, 1]],
            z=[embeddings[i, 2], embeddings[i + 1, 2]],
            mode='lines',
            line=dict(color=color, width=4),
            showlegend=False
        )
        lines.append(line)
    return lines

# Create 3D scatter plot for player 1 and player 2
trace1 = go.Scatter3d(
    x=reduced_embeddings_1[:, 0],
    y=reduced_embeddings_1[:, 1],
    z=reduced_embeddings_1[:, 2],
    mode='markers+text',
    marker=dict(size=4, color='blue'),
    text=words_player1,
    name='Player 1'
)

trace2 = go.Scatter3d(
    x=reduced_embeddings_2[:, 0],
    y=reduced_embeddings_2[:, 1],
    z=reduced_embeddings_2[:, 2],
    mode='markers+text',
    marker=dict(size=4, color='red'),
    text=words_player2,
    name='Player 2'
)

if words_player2[-1] == words_player1[-1]:
    print("won")
    # Add the last point as a star for Player 2
    last_point_player = go.Scatter3d(
        x=[reduced_embeddings_2[-1, 0]],
        y=[reduced_embeddings_2[-1, 1]],
        z=[reduced_embeddings_2[-1, 2]],
        mode='markers+text',
        marker=dict(size=8, color='green', symbol="diamond-open"),
        text=[words_player2[-1]],
        name='Final word'
    )


# Create gradient lines for both players using Plotly colormaps
lines_player1 = create_px_colormap_gradient_lines(reduced_embeddings_1, len(words_player1), 'Plotly3')
lines_player2 = create_px_colormap_gradient_lines(reduced_embeddings_2, len(words_player2), 'Hot')

# Combine all traces
data = [trace1, trace2] + lines_player1 + lines_player2
if words_player2[-1] == words_player1[-1]:
    data = data + [last_point_player]
# Define layout with larger figure size
layout = go.Layout(
    title='3D Scatter Plot with Colormap Gradient Lines',
    scene=dict(
        xaxis=dict(title='PCA1'),
        yaxis=dict(title='PCA2'),
        zaxis=dict(title='PCA3'),
    ),
    legend_title_text='Players',
    width=800,
    height=700,
)

# Create figure and show
fig = go.Figure(data=data, layout=layout)
fig.show()